<a href="https://colab.research.google.com/github/plaban1981/Huggingface_transformers_course/blob/main/Huggingface_transformers_pipeline_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  creating and using a model

* The **AutoModel** class and all of its relatives are actually simple wrappers over the wide variety of models available in the library.
* The AutoModel wrapper can automatically guess the appropriate model architecture for your checkpoint, and then instantiates a model with this architecture.
* In case we know the type of model we want to use, you can use the class that defines its architecture directly.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 19.7MB/s 
     |████████████████████████████████| 3.3MB 41.5MB/s 
     |████████████████████████████████| 901kB 30.8MB/s 
